# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config_Prince import gmaps_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# indentify source
file_source = "../Resources/City_Data.csv"

# load the sourse as a dataframe
city_data_df = pd.read_csv(file_source)
city_data_df.head()

,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,41.00,100,90,8.05,CL,1619803545
1,Esperance,-33.8667,121.9000,46.40,93,72,5.75,AU,1619803545
2,Lichinga,-13.3128,35.2406,68.00,83,20,6.91,MZ,1619803546
3,Tabory,58.5198,64.5495,37.22,49,41,7.36,RU,1619803546
4,Kaitangata,-46.2817,169.8464,48.00,72,41,4.00,NZ,1619803546


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# identify variables to be used for headmap
locations = city_data_df[['lat','Lng']]
humidity = city_data_df['Humidity']

In [8]:
#plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add the layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
hotel_df = city_data_df.loc[(city_data_df['Max Temp']>70) &
                            (city_data_df['Max Temp']<80)&
                          (city_data_df['Wind Speed']<10)&
                          (city_data_df['Cloudiness']==0),:
]
# hotel_df['Hotel Name'] = ""
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df

,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,77.00,78,0,4.61,RE,1619803598
1,Guerrero Negro,27.9769,-114.0611,78.19,37,0,8.90,MX,1619803599
2,Saint-Pierre,-21.3393,55.4781,77.00,78,0,4.61,RE,1619803614
3,Kumul,42.8000,93.4500,74.46,13,0,1.90,CN,1619803625
4,Gönen,40.1049,27.6540,75.20,72,0,4.61,TR,1619803633
5,Morón,-34.6534,-58.6198,75.00,49,0,5.01,AR,1619803665
6,Laranjeiras do Sul,-25.4078,-52.4161,73.71,33,0,5.66,BR,1619803680
7,Soltvadkert,46.5789,19.3939,71.60,46,0,4.61,HU,1619803732
8,Usakos,-22.0000,15.6000,75.07,20,0,8.97,NaN,1619803735


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
# find the Hotels restaurant of each type to coordinates  hotel_df.iloc[:,1:2],lodging
hotel_names = []
x = 0

In [70]:
# for loop to capture hotels out of city
for cities in hotel_df['City']:
    target_cord = str(hotel_df['lat'][x])+' , '+str(hotel_df['Lng'][x])
    x = x + 1
    radius = 5000
    types = 'lodging'
    params = {
    'location': target_cord,    
    "radius": radius,
    "type": types,
    "key": gmaps_key,
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    #try/except to capture the data
    try:
        response = requests.get(base_url,params=params)
        results = response.json()
        hotel_names.append(results['results'][0]['name'])
        
    except IndexError:
        hotel_names.append(np.nan)
        
# hotel_df["Hotel Names"] = hotel_names
# hotel_df

In [75]:
hotel_df["Hotel Names"] = hotel_names
hotel_df
# hotel_df.head()

C:\Users\14343\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Names
0,Saint-Philippe,-21.3585,55.7679,77.00,78,0,4.61,RE,1619803598,"Chambres d'hôte ""La Trinité"""
1,Guerrero Negro,27.9769,-114.0611,78.19,37,0,8.90,MX,1619803599,"Casa Laguna, Bed & Breakfast"
2,Saint-Pierre,-21.3393,55.4781,77.00,78,0,4.61,RE,1619803614,Lindsey Hôtel
3,Kumul,42.8000,93.4500,74.46,13,0,1.90,CN,1619803625,Laoyutang Resort
4,Gönen,40.1049,27.6540,75.20,72,0,4.61,TR,1619803633,Gönen Palmiye Butik Pansiyon
5,Morón,-34.6534,-58.6198,75.00,49,0,5.01,AR,1619803665,Hotel Morón
6,Laranjeiras do Sul,-25.4078,-52.4161,73.71,33,0,5.66,BR,1619803680,Hotel Palmeiras
7,Soltvadkert,46.5789,19.3939,71.60,46,0,4.61,HU,1619803732,Frittmann Borászat és Panzió
8,Usakos,-22.0000,15.6000,75.07,20,0,8.97,NaN,1619803735,Hotel Pension Bahnhof


In [83]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [81]:
locations = hotel_df[["lat", "Lng"]]

In [82]:
# Add marker layer ontop of heat map

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…